resize selenium window?

In [ ]:

var rows = 6;
var columns = 6;
var screen = {
    height: 1024 * 6,
    width: 1350 * 6
};

function getScreenSize() {
    return client
        .windowHandle()
        .then(r => client.windowHandleMaximize(r.value))
        .catch(e => {})
        .windowHandleSize()
        .then(r => (screen = r.value))
        .catch(e => console.log(e))
}

function resizeWindow(i = 0) {
    i = Math.abs(i + (typeof OFFSET !== 'undefined' ? OFFSET : 0));
    var hwnd;
    return client
        .windowHandle()
        .then(r => client.window((hwnd = r.value)))
        .then(() => typeof screen === 'undefined' ? getScreenSize() : screen)
        .catch(e => console.log(e))
        .then(screen => {
            const newPosition = {
                x: Math.floor(i % columns)
                    * Math.floor(screen.width / columns),
                y: Math.floor(i / columns % rows)
                    * Math.floor(screen.height / rows)
            };
            console.log(newPosition);
            return client.windowHandlePosition(hwnd, newPosition)
        })
        .catch(e => console.log(e))
        .then(() => client.windowHandleSize(hwnd, {
            width: Math.floor(screen.width / columns),
            height: Math.floor(screen.height / rows)
        }))
        .catch(e => console.log(e))
}

if(typeof client.resizeWindow === 'undefined') {
    client.addCommand('resizeWindow', resizeWindow);
}
module.exports = resizeWindow;


only one window?



In [ ]:
var importer = require('../Core');

function closeAllTabs(keep) {
    return client
        .getTabIds()
        .then(h => {
            var promises = [];
            for(var i = 0; i < h.length; i++) {
                if(h[i] === keep) {
                    continue;
                }
                promises.push((i => resolve => client
                              .switchTab(h[i])
                              .close()
                              .then(() => resolve())
                              .catch(e => resolve()))
                              .apply(this, [i]));
            }
            return importer.runAllPromises(promises);
        })
        .catch(e => console.log(e))
}

function closeAllWindows(keep) {
    return client
        .windowHandles()
        .then(h => {
            var promises = [];
            for(var i = 0; i < h.value.length; i++) {
                if(h.value[i] === keep) {
                    continue;
                }
                promises.push((i => resolve => client
                              .window(h.value[i])
                              .close()
                              .then(() => resolve())
                              .catch(e => resolve()))
                              .apply(this, [i]));
            }
            return importer.runAllPromises(promises);
        })
        .window(keep)
        .catch(e => console.log(e))
}

function onlyOneWindow() {
    return client
        .newWindow('https://google.com')
        .windowHandle()
        .then(r => closeAllWindows(r.value))
        .getCurrentTabId()
        .then(r => closeAllTabs(r))
        .catch(e => console.log(e))
    // TODO: close all tabs
}

module.exports = onlyOneWindow;



tile selenium chrome windows?



In [ ]:
var importer = require('../Core');
var {getAllSessionUrls} = importer.import(['resize selenium window', 'get all session urls'], {client})

function openUrl(url) {
    var handles;
    return client
        .windowHandles()
        .then(h => handles = h.value)
        .execute((url, width, height) => {
            window.open(url, '', 'width=1350,height=1024');
        }, url)
        .windowHandles()
        .then(h => h.value.filter(hwnd => handles.indexOf(hwnd) === -1)[0])
        .catch(e => console.log(e))
}

function createNewWindows(urls) {
    var hwnd;
    return client
        .windowHandle()
        .then(r => hwnd = r.value)
        .windowHandles()
        .then(r => {
            // -1 first window is already used by sliced url
            const count = urls.length - (r.value.length - 1);
            const promises = [];
            for(var i = 0; i < count; i++) {
                promises.push((i => resolve => {
                    console.log(urls[i]);
                    return openUrl(urls[i])
                        .window(hwnd)
                        .then(h => resolve(h))
                        .catch(e => { console.log(e); resolve() })
                }).apply(this, [i]));
            }
            return importer.runAllPromises(promises);
        })
        .catch(e => console.log(e))
}

function tileWindows(urls) {
    return client
        .resizeWindow(0)
        .url(typeof urls === 'string' ? urls : urls[0])
        .then(h => typeof urls === 'string' ? [] : createNewWindows(urls.slice(1)))
        .then(() => getAllSessionUrls())
        .windowHandles()
        .catch(e => console.log(e))
}
module.exports = tileWindows;



In [ ]:
var importer = require('../Core');
var runSeleniumCell = importer.import('selenium cell');

var query = 'list of search engines';

$$.async();
// get multiple sets of 9 working for single page scraping in parallel
runSeleniumCell('tile chrome windows')
    .then(tileWindows => tileWindows([
        'https://www.google.com/search?q=' + query,
        'https://www.bing.com/search?q=' + query,
        'https://search.yahoo.com/search?p=' + query,
        'https://www.ask.com/web?q=' + query,
        'https://search.aol.com/aol/search?q=' + query,
        'https://www.baidu.com/s?wd=' + query,
        'https://www.wolframalpha.com/input/?i=' + query,
        'https://duckduckgo.com/?q=' + query,
        'https://archive.org/search.php?query=' + query,
    ]))
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))



get all session and window urls?



In [ ]:
var importer = require('../Core');

function getAllSessionUrls(reposition = true) {
    var windowCounter = -1;
    var session = client.requestHandler.sessionID;
    return client
        .getActiveSessions()
        .then(r => {
            console.log(r);
            return importer.runAllPromises(r.map(s => resolve => {
                client.requestHandler.sessionID = s;
                return client
                    .getUrl()
                    .windowHandles()
                    .then(h => importer.runAllPromises(h.value.map(hwnd => resolve => {
                        windowCounter++
                        return client
                            .window(hwnd)
                            .then(() => reposition ? resizeWindow(windowCounter) : [])
                            .getUrl()
                            .then(r => resolve(r))
                            .catch(e => console.log(e))
                    })))
                    .then(r => resolve(r))
                    .catch(e => console.log(e))
            }))
        })
        .then(r => {
            client.requestHandler.sessionID = session;
            return r;
        })
        .catch(e => console.log(e))
}

module.exports = getAllSessionUrls;


In [ ]:
var importer = require('../Core');
var runSeleniumCell = importer.import('selenium cell');

$$.async();
runSeleniumCell(['resize selenium window', 'get all session urls'], false)
    .then(r => r.getAllSessionUrls(false))
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))


send a joke?

In [12]:
function sendJoke(hwnd, joke) {
    return client
        .window(hwnd)
        .then(() => sendFacebookMessage(joke[0]))
        .pause(30000)
        .then(() => sendFacebookMessage(joke[1]))
}
module.exports = sendJoke;


[Function: sendJoke]

Monitor many chats at once and continuously


In [ ]:
var util = require('util');
var request = util.promisify(require('request'));
var importer = require('../Core');
var runSeleniumCell = importer.import('selenium cell');

var query = 'list of search engines';

function regexToArray(ex, str, i = 0) {
    var co = []; var m;
    while ((m = ex.exec(str)) && co.push(m[i]));
    return co;
}

var jokes;
function getJoke() {
    return (typeof jokes === 'undefined'
        ? request('http://www.ducksters.com/jokes/silly.php')
        .then(res => regexToArray(/^\s*Q:.*$|^\s*A:.*$/igm, res.body))
        : Promise.resolve(jokes))
        .then(arr => {
            const i = Math.round(Math.random() * arr.length / 2) * 2;
            return arr.slice(i, i + 2);
        })
}

var tileWindows, jokes, windows, sendFacebookMessage;

var friends = [
    'https://www.facebook.com/messages/t/jaimeneufer',
    'https://www.facebook.com/messages/t/nathaniel.ellingson.3',
/*
    'https://www.facebook.com/messages/t/aneesh.karve',
    'https://www.facebook.com/messages/t/robert.chandler',
    'https://www.facebook.com/messages/t/robert.holweger',
    'https://www.facebook.com/messages/t/keifer.street',
    /*
    'https://www.facebook.com/messages/t/ragunr',
    'https://www.facebook.com/messages/t/johan.larson.56',
    'https://www.facebook.com/messages/t/abe.pralle',
    'https://www.facebook.com/messages/t/wolfdieter.otte',
    'https://www.facebook.com/messages/t/carisa.knight6000',
    'https://www.facebook.com/messages/t/talbert.tso',
    'https://www.facebook.com/messages/t/abe.anwary',
    'https://www.facebook.com/messages/t/devin.west2',
    'https://www.facebook.com/messages/t/eurostar88',
    */
];

function tellJokes() {
    // get multiple sets of 9 working for single page scraping in parallel
    return runSeleniumCell([
        'tile chrome windows',
        'log in facebook',
        'send facebook message',
        'send a joke'
    ])
        .then(r => {
            tileWindows = r.tileWindows;
            sendFacebookMessage = r.sendFacebookMessage;
            sendJoke = r.sendJoke;
            return r.loginFacebook()
        })
        .then(() => tileWindows(friends))
        .then(windows => {
            return importer.runAllPromises(windows.value.map(hwnd => resolve => {
                return getJoke()
                    .then(joke => sendJoke(hwnd, joke).then(() => resolve()))
            }))
        })
        .catch(e => console.log(e))
}

$$.async();
tellJokes()
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))


Initializing webdriver on localhost
[ 1510460569568, '6d7f6145a1776b200d1fc5abe9a88dd5' ]
[21:25:21]  COMMAND	GET 	 "/wd/hub/session/6d7f6145a1776b200d1fc5abe9a88dd5/window_handle"
[21:25:21]  COMMAND	POST 	 "/wd/hub/session/6d7f6145a1776b200d1fc5abe9a88dd5/window"
[ 1510460574522, '187c5a0e55072a91354398bd7a74fb9a' ]
[21:25:21]  COMMAND	GET 	 "/wd/hub/session/187c5a0e55072a91354398bd7a74fb9a/window_handle"
[21:25:21]  COMMAND	POST 	 "/wd/hub/session/187c5a0e55072a91354398bd7a74fb9a/window"
[21:25:21]  COMMAND	GET 	 "/wd/hub/session/187c5a0e55072a91354398bd7a74fb9a"
[ 1510460574768, '707b5f83330038abf6d6b77c41f01fca' ]
[21:25:21]  COMMAND	GET 	 "/wd/hub/session/707b5f83330038abf6d6b77c41f01fca/window_handle"
[21:25:21]  COMMAND	POST 	 "/wd/hub/session/707b5f83330038abf6d6b77c41f01fca/window"
[21:25:21]  COMMAND	GET 	 "/wd/hub/session/707b5f83330038abf6d6b77c41f01fca"
[ 1510460672818, 'ae4480a26f62f1bde47f9ced12c2b291' ]
[21:25:21]  COMMAND	GET 	 "/wd/hub/session/ae4480a26f62f1bde47f9ce

[21:25:33]  COMMAND	POST 	 "/wd/hub/session/187c5a0e55072a91354398bd7a74fb9a/keys"


[21:25:33]  COMMAND	POST 	 "/wd/hub/session/187c5a0e55072a91354398bd7a74fb9a/keys"


[21:25:34]  COMMAND	POST 	 "/wd/hub/session/187c5a0e55072a91354398bd7a74fb9a/keys"
[21:26:05]  COMMAND	POST 	 "/wd/hub/session/187c5a0e55072a91354398bd7a74fb9a/element"
[21:26:05]  COMMAND	POST 	 "/wd/hub/session/187c5a0e55072a91354398bd7a74fb9a/element/0.16074398324069472-1/click"


[21:26:05]  COMMAND	POST 	 "/wd/hub/session/187c5a0e55072a91354398bd7a74fb9a/keys"


[21:26:05]  COMMAND	POST 	 "/wd/hub/session/187c5a0e55072a91354398bd7a74fb9a/keys"


[21:26:05]  COMMAND	POST 	 "/wd/hub/session/187c5a0e55072a91354398bd7a74fb9a/keys"
[21:26:06]  COMMAND	POST 	 "/wd/hub/session/187c5a0e55072a91354398bd7a74fb9a/window"
[21:26:06]  COMMAND	POST 	 "/wd/hub/session/187c5a0e55072a91354398bd7a74fb9a/element"
[21:26:06]  COMMAND	POST 	 "/wd/hub/session/187c5a0e55072a91354398bd7a74fb9a/element/0.07077395637861361-1/click"


[21:26:07]  COMMAND	POST 	 "/wd/hub/session/187c5a0e55072a91354398bd7a74fb9a/keys"


[21:26:07]  COMMAND	POST 	 "/wd/hub/session/187c5a0e55072a91354398bd7a74fb9a/keys"


[21:26:08]  COMMAND	POST 	 "/wd/hub/session/187c5a0e55072a91354398bd7a74fb9a/keys"


clean up old selenium sessions?

In [ ]:
var importer = require('../Core');
var execCmd = importer.import('spawn child process');
var runSeleniumCell = importer.import('selenium cell');
var createWebdriverClient = importer.import('webdriver client');

var regexToArray = (ex, str, i = 0) => {
    var co = []; var m; while ((m = ex.exec(str)) && co.push(m[i])); return co;
};

$$.async();
var client, sessions;
execCmd('docker logs act-selenium')
    .then(r => regexToArray(/\/session\/(.+?)\//ig, r.join('\n'), 1))
    .then(r => sessions = r.filter((s, i, arr) => arr.indexOf(s) === i))
    .then(() => client = createWebdriverClient())
    .then(() => {
        return importer.runAllPromises(sessions.map(session => (resolve) => {
            client.requestHandler.sessionID = session;
            client
                .windowHandle()
                .then(r => client.window(r.value))
                .session()
                .end()
                .then(s => resolve(s.sessionId))
                .catch(e => resolve())
        }))
    })
    .then(r => $$.sendResult(r))
    .catch(r => $$.sendError(r))

